In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11.7,6)})
os.getcwd()

'C:\\Users\\tomdx\\Documents\\fedex'

In [2]:
os.chdir('C:\\Users\\tomdx\\Documents\\fedex')

In [3]:
df = pd.read_excel('data_fed.xlsx',sheet_name='Raw Data')

In [4]:
df.isnull().sum()

Customer Number                0
Customer Country               0
Year                           0
Month                          0
Shipment Destination Region    0
Revenue (EUR)                  0
dtype: int64

In [5]:
df['Month'] = df['Month'].astype(str)
df['month'] = df['Month'].str[-2:].astype(int)
df.loc[(df['month']>=1) & (df['month']<=3),'quarter'] = 'Q1'
df.loc[(df['month']>=4) & (df['month']<=6),'quarter'] = 'Q2'
df.loc[(df['month']>=7) & (df['month']<=9),'quarter'] = 'Q3'
df.loc[(df['month']>=10) & (df['month']<=12),'quarter'] = 'Q4'


In [6]:
quarterly_segmented = df.groupby(['Customer Number','Year','quarter','month'])['Revenue (EUR)'].sum().reset_index()

In [7]:
jf = quarterly_segmented
jf = jf.groupby('Customer Number')['Revenue (EUR)'].pct_change()*100
jf = jf.reset_index().drop(columns=['index']).rename(columns={'Revenue (EUR)':'pct_chng'})

In [8]:
quarterly_segmented = quarterly_segmented.join(jf)
quarterly_segmented.fillna(0,inplace=True)


In [9]:
buy_2018 = quarterly_segmented.loc[(quarterly_segmented['Year']==2018)] 
buy_q3_2018 = buy_2018[buy_2018['quarter']=='Q3'] 
buy_q3_2018['last Q'] = 'Y'

<ipython-input-9-8d1260952a77>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buy_q3_2018['last Q'] = 'Y'


In [10]:
ff = pd.merge(df[['Customer Number']],buy_q3_2018,on=['Customer Number'],how='left').drop_duplicates()
ff[ff['last Q']=='Y'].nunique()
ff['last Q'] = ff['last Q'].fillna('N')
 

In [11]:
ff = ff.drop_duplicates(subset='Customer Number')
  

In [12]:
df.loc[df['Customer Number'].isin(buy_q3_2018['Customer Number']),'Bought on last Q'] = "Y" 
df.loc[(df['Bought on last Q']!='Y'),'Bought on last Q'] = 'N'

In [13]:
bought_on_q3_2018 = pd.DataFrame(buy_q3_2018['Customer Number'].unique())

In [14]:
gg =  df.groupby(['Customer Number','Year','quarter'])['Revenue (EUR)'].sum().reset_index()
ff = gg[~((gg['Year'] == 2018) & (gg['quarter']=='Q3'))]
 

In [15]:
grouped = df[df['Bought on last Q']=='Y'].groupby(['Customer Number','Year','quarter'])['Revenue (EUR)'].sum().reset_index() 

In [16]:
num_orders = grouped['Customer Number'].value_counts().reset_index().rename(columns={'index':'Customer Number','Customer Number':'quarter count'})

In [17]:
grouped = pd.merge(grouped,num_orders,on='Customer Number')
 

In [18]:
grouped.loc[grouped['quarter count']==1,'Only Last Q'] = 'Y' 

In [19]:
only_last_q = grouped.loc[(grouped['quarter count']==1) & (grouped['Year']==2018) & (grouped['quarter']=='Q3')] 
 

In [20]:
df.loc[(df['Customer Number'].isin(only_last_q['Customer Number'])),'Only Last Q'] = 'Y' 

In [21]:
df['Only Last Q'] = df['Only Last Q'].fillna('N')

In [22]:
q3_q2_2018 = df.loc[(df['Year']==2018)  & ((df['quarter']=='Q3')|(df['quarter']=='Q2'))]
q3_2018 = df.loc[(df['Year']==2018)  & ((df['quarter']=='Q3'))]

not_q3_2018 = df.loc[~((df['Year']==2018)  & ((df['quarter']=='Q3')))]
not_q3_2018 = not_q3_2018.groupby('Customer Number')['Revenue (EUR)'].sum().reset_index().rename(columns={'Revenue (EUR)':'Last 6 Q Revenue'})
q3_2018 = q3_2018.groupby('Customer Number')['Revenue (EUR)'].sum().reset_index().rename(columns={'Revenue (EUR)':'Last Q Revenue'})


In [23]:
rest_of_grouped =  df.loc[~((df['Year']==2018)  & ((df['quarter']=='Q3')|(df['quarter']=='Q2')))]

In [24]:
last_2q_revenue = q3_q2_2018.groupby('Customer Number')['Revenue (EUR)'].sum().reset_index().rename(columns={'Revenue (EUR)':'Last 2 Q Revenue'})
 

In [25]:
first_q_revenues = rest_of_grouped.groupby('Customer Number')['Revenue (EUR)'].sum().reset_index().rename(columns={'Revenue (EUR)':'First 5 Q Revenue'})
 

In [26]:
agg_quarterly_revenue = pd.merge(first_q_revenues,last_2q_revenue,how='outer').fillna(0)
agg_quarterly_revenue = pd.merge(agg_quarterly_revenue,not_q3_2018,how='outer').fillna(0)
agg_quarterly_revenue = pd.merge(agg_quarterly_revenue,q3_2018,how='outer').fillna(0)

 

In [27]:
agg_quarterly_revenue['avg_qtrly_rev_5'] = agg_quarterly_revenue['First 5 Q Revenue']/5
agg_quarterly_revenue['avg_qtrly_rev_2'] = agg_quarterly_revenue['Last 2 Q Revenue']/2

In [28]:
 agg_quarterly_revenue['pct_chng'] = round((agg_quarterly_revenue['avg_qtrly_rev_2'] - agg_quarterly_revenue['avg_qtrly_rev_5'])/agg_quarterly_revenue['avg_qtrly_rev_2'],4)*100

In [29]:
agg_quarterly_revenue['pct_chng'] = agg_quarterly_revenue['pct_chng'].replace(-np.inf, np.nan) 

In [30]:
agg_quarterly_revenue['pct_chng'].mean() 

-233.2913903743317

In [31]:
agg_quarterly_revenue['Customer Number'].nunique()

647

In [32]:
agg_quarterly_revenue.loc[(agg_quarterly_revenue['pct_chng'] < \
                           agg_quarterly_revenue['pct_chng'].median()),'Segment'] = 'Down-Trader'

In [33]:
agg =  agg_quarterly_revenue.copy()

agg.loc[(agg['Last 6 Q Revenue']!=0.00) & (agg['Last Q Revenue']==0.00),'Segment'] = 'Lost Customer'
agg.loc[(agg['pct_chng'] < agg['pct_chng'].median()),'Segment'] = 'Down-Trader'
agg.loc[(agg['pct_chng'] > 0.00),'Segment'] = 'Growth Customers'
agg.loc[(agg['Segment'].isnull()) & (agg['pct_chng'] >= -20),'Segment'] = "'On the verge' Customers"
agg.loc[(agg['Segment'].isnull()) & (agg['pct_chng'] <= -20),'Segment'] = "Down-Trader"
agg.loc[(agg['First 5 Q Revenue']==0.00) & (agg['Last 2 Q Revenue']!=0.00),'Segment'] = 'Newly Acquired'


In [34]:
#agg.to_csv('segmented.csv',index=0)

In [35]:
agg.head()

,Customer Number,First 5 Q Revenue,Last 2 Q Revenue,Last 6 Q Revenue,Last Q Revenue,avg_qtrly_rev_5,avg_qtrly_rev_2,pct_chng,Segment
0,ABC0000464,5477.80,812.72,6131.32,159.20,1095.560,406.360,-169.60,Down-Trader
1,ABC0009734,5634.44,1058.10,6406.01,286.53,1126.888,529.050,-113.00,Down-Trader
2,ABC0012173,4522.23,1123.22,5345.61,299.84,904.446,561.610,-61.05,Down-Trader
3,ABC0026905,6993.66,2196.53,8095.81,1094.38,1398.732,1098.265,-27.36,Down-Trader
4,ABC0027135,5161.58,1499.32,6196.27,464.63,1032.316,749.660,-37.70,Down-Trader
